# MIE1624 - Assignment 2
## Bibin K. Sebastian
## 1003752691

In [ ]:
#Packages that may need PIP install 
!pip install missingno 
!pip install mlxtend 
!pip install seaborn 

In [6]:
# Import required Libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np 
import seaborn as sns
#plt.style.use('seaborn-whitegrid')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score ,r2_score,explained_variance_score
from sklearn.preprocessing import LabelEncoder 
import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, auc, mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold ,StratifiedKFold

import missingno as msno # for visualizing the Nans in dataset

from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt

cur_dir = os.getcwd()

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.linear_model import Lasso
import pprint
pp = pprint.PrettyPrinter(indent=4)
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [7]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


Using TensorFlow backend.


In [8]:
# read the survey results as a Data Frame
df = pd.read_csv('Kaggle_Salary.csv',)

In [ ]:
df.shape

# Creating a Data dictionary should be the first step and my dictionary is as follows . 

![DD](dd.png)

# Higher level visualisations to get an overview of the data

### Lets see a brief outlook of the missing data as heat map

In [ ]:
msno.matrix(df) # This will create a heatmap similar seaborn heat map . Black values are not NaNs white area is NaN
plt.title("Missing values in the Data Frame",fontsize=18)
plt.xlabel("Features from 0-395",fontsize=16)
plt.ylabel("Survey entries from 1 - 15429",fontsize=16)

** We can see that from about 10000 rows, many rows are sparse towards the right side of the features. However, most of the first 30 columns are filled in which means we need not drop these rows. These rows need to be closely inspected ** 

In [ ]:
#read the Multiple Choice Responses
MCR_df = pd.read_csv("Kaggle_Salary.csv")

In [ ]:
MCR_df.pop('Unnamed: 0'); # Remove the additional index that cropped up while reading the dataframe
MCR_df.pop('index');# Remove the extra index column
y =MCR_df.pop('Q9'); #get the 

In [ ]:
#drop all free Form Responses (including the duration/ Time to fill the answers)
MCR_df.drop([col for col in MCR_df.columns if col.find("TE")>0 or col.find("seconds")>0],axis=1,inplace=True)

In [ ]:
#Replces columns format with the actual survey question for visualization purposes
MCR_df.columns = [(col.split(" - ",1)[0] # split the questions text from first row at the '-' and choose the first part
                   .split(" (Select",1)[0] # some questions have extra sentences starts with 'Select'
                   .split(" (include",1)[0]# some questions have extra sentences starts with 'Include' and 'Answers'
                   .split(" (Answers",1)[0]) for col in MCR_df.loc[0].values]

# compiles all the unique question columns
questions = []
[questions.append(item) for item in MCR_df.columns if item not in questions] # remove all the duplicate entries 

#drops the extra row for the question columns
MCR_df.drop(0,axis=0,inplace=True)

In [ ]:
questions[:10] # check if the questions were retrieved correctly

In [ ]:
MCR_df.shape

In [ ]:
# Creating a function to automate the plotting of all the 50 Questions . The function takes start and end index of the questions

def data_plot(start_index,end_index):
    # loop for plotting the responses from question #1 to question #50
    for i,question in zip(range(start_index,end_index,1),questions[start_index : end_index]):
        # prepares the data for plotting. melt is used for questions represented with multiple columns
        if len(MCR_df[question].shape)==1: 
            data = MCR_df[question]
        else: 
            data = MCR_df[question].melt()['value']

        # plot specifications
        plt.figure(figsize=(15,6.5))
        (data
         .str.wrap(35) # text wrap for long sentences
         .value_counts(sort=True)[:15]# counts then sorts unique values and limits the number of y labels to 15 (to fit in the graph)
         .plot.bar(fontsize=20) # plotting of the bar chart

         # sets the plot title + a short interpretation
         .set_title("\n\nQ"+str(i)+" :"+pd.Series(question).str.wrap(65)[0]+
                    '\n Top answer: "'+ str(data.value_counts(sort=True).head(1).index[0])+
                    '"\n with ' + str(data.value_counts(sort=True).head(1)[0]) + " responses \n",
                    fontdict={'fontsize':22})
        )
        plt.ylabel('Count',fontsize=20)
        plt.show()

In [ ]:
# Lets visualize the 50 Questions to get an overview of the distribution of Q & A 
data_plot(0,30) 

In [ ]:
data_plot(31,50) # remaining plots 

**We get a very high level overview from these plots such as**

1. This survey is skewed towards male population! we need to work towards gender equality 
2. USA & INDIA has maximum participants in this survey 
3. most of the participants are around 25years old
4. Almost half of the participants are from Computer Scinence background
5. Most Kaggers are either Data Scientist or Student (Aspiring Data Scientists)
6. Main industry is Computer Technology
7. Coursera was mentioned as the top MOOC platform ( I agree wholeheartedly - Major user of Coursera)
8. Python is the Go to language for most of us 
9. Many ML models are difficult to understand and interpret 
*


Good insights to know and work on !! lets go deeper 



# DATA Cleaning and Subsetting 

**Dropping the description of the columns as it is interfering with the further processes. <br> I have popualated a Data Dictionary as below**

In [ ]:
df1 = df.drop(df.index[0]) # drop the first row containing the questions as it was interfering with the 

In [ ]:
df1.shape # looks right

In [ ]:
df1.head()

In [ ]:
df1.pop('Unnamed: 0'); # getting rid of the pesky pseudo index column "Unnamed: 0"

In [ ]:
# there are a number of columns that has other text in it which showed that the respondants typed responses on their own
# even though I believe, there is some information in these columns , I doubt it will be useful since it will blow up 
# the number of features when onehot encoded . So lets get rid of them for now 

othertext=[]
for head in df1.columns:
    if 'TEXT' not in head :
        continue
    else:
        othertext.append(head)
othertext       

In [ ]:
df1.shape


In [ ]:
df2 = df1.drop(labels=othertext, axis =1)
df2.pop('Q32_OTHER');
# saving the data frame to another data frame df2 after dropping the colums
# that contain other text 

In [ ]:
df2.shape

In [ ]:
df2.head(3)

In [ ]:
df2.pop('index'); # removing the index column 

In [ ]:
df2.rename(columns={'Time from Start to Finish (seconds)':'Time'},inplace=True) # renaming time from start to finish to TIme 

### Saving the slightly cleaned dataframe to a csv so that when we read it again, the data types will be read in properly . earlier ,due to the second row of questions, pd read csv function could not interpret all the data types properly 

In [ ]:
df2.to_csv('Initial_cleaned.csv')  # Write to csv so that we can read properly 

In [ ]:
df3 = pd.read_csv('Initial_cleaned.csv',)

In [ ]:
df3.pop('Unnamed: 0'); # getting rid of the pesky pseudo index column "Unnamed: 0"

In [ ]:
df1.dtypes.value_counts() # earlier since all the columns had a question row, pandas could not infer the data types properly  

In [ ]:
df3.dtypes.value_counts() # this looks much more reasonable

In [ ]:
# checking the float columns
float_col = []
for i,col in enumerate(df3.columns):
    if df3[col].dtype == 'float64':
        float_col.append(i)
df3.iloc[:,float_col].head()

In [ ]:
# Looking at higer level outliers
df3[(df3['Q9']==500000) & (df3.Q7 == 'I am a student')]

In [ ]:
df3[(df3['Q9']==500000) & (df3.Q7 == 'I am a student')].shape # These rows are clear corrupt data. None is 21 years old 
# and has 30+ years experience and earns 500k USD !!  I wish I could do that :-)

In [ ]:
df3 = df3.drop(df3[(df3['Q9']==500000) & (df3.Q7 == 'I am a student')].index) # dropping the 6 rows straight away - 
#explanation given above

In [ ]:
df3.reset_index(drop=True,inplace=True)

In [ ]:
df3.shape

In [ ]:
df3.head()

In [ ]:
df3.dtypes.value_counts() # most of the columns are object type 

In [ ]:
df3.loc[:,df3.dtypes.values == "int64"].head() # only time and Q9 is shown as integer I think we have more 

In [ ]:
df3.loc[:,df3.dtypes.values == "float64"].head() #  

# Finding a good subset to conduct regression and predictions 


In [ ]:
df3.Q7.value_counts(dropna=False).plot(kind='bar')
plt.xlabel("Industries",fontsize = 16)
plt.ylabel("Count",fontsize = 16)
plt.title("Distribution of Job Industries",fontsize = 20)

## Objective : Since I am graduating soon, I am quite interested in finding out more about the possible industries that I am aiming to get a job in .  I take this assignment as a good opportunity to do so and I have picked three industries that I am interested in . <br>
1.Computers/Technology <br>
2.Academics/Education <br>
3.Accounting/Finance<br>

In [ ]:
df3_job = df3[df3.Q7.isin(['Computers/Technology','Academics/Education','Accounting/Finance'])]

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(df3_job.Q7)
locs, labels = plt.xticks()
plt.setp(labels, rotation=45)
plt.title("Distribution of Industries in the chosen Data Frame")
plt.ylabel('Count', fontsize = 18)
plt.xlabel('Industries', fontsize = 18)

In [ ]:
industry = np.array(df3_job.Q7.value_counts().index)
industry

## Lets have a look at the median salaries for these industries 
Academics Median salary looks to be lower. It may be because of the fact that in some countries like India and China, academicians tend to get lower salary 

In [ ]:
# Check this out 

#https://stackoverflow.com/questions/7526625/matplotlib-global-legend-and-title-aside-subplots

In [ ]:
for idx,ind in enumerate(industry): 
    figs={}
    axs={}
    figs[idx]=plt.figure(figsize=(10,8)) # create a figure object
    axs[idx]=figs[idx].add_subplot(211) # Add a subplot
    axs[idx] = sns.boxplot(df3_job['Q9'][((df3_job.Q7 == ind) & df3_job.Q9.between(left=10000,right=250000))],palette='Set2')
    plt.title(ind)
    plt.xlabel("Annual Salary")    

In [ ]:
# lets see how the salary distribution of each industries are across top countries 
for idx1 , ind in enumerate(industry):
    fig = plt.figure(figsize=(35,25))
    plt.xlabel(ind)
    for idx2 , country in enumerate(df3_job.Q3.value_counts(sort=True).index[:3]):
        plt.subplot(4,5,idx2+1)
        df3_job[((df3_job['Q3'] == country) & (df3_job['Q7'] == ind)) ].Q9.hist(stacked=True)
        plt.title(country,fontsize = 20)
        plt.ylabel(ind,fontsize = 18)
        plt.xlabel("Annual Salary")
    plt.tight_layout()
    plt.show()



### We could clearly see that United States of America has a much spread out distribution as well as the highest mean salary amoung top 3 countries. The feature "Which country you are coming from?" certainly plays a major role in the salary decision. We will check this out during the feature importance step

# Exploring each question and cleaning them as needed

### cleaning outliers and data that clearly looks flawed

In [ ]:
# looking at very high salary and very low experience to check for anomalies
df3_job[((df3_job['Q9']>300000)  & (df3_job.Q8 == '0-1') & (df3_job.Q6 != 'Chief Officer'))]

In [ ]:
index_drp = df3_job[((df3_job['Q9']>300000)  & (df3_job.Q8 == '0-1') & (df3_job.Q6 != 'Chief Officer'))].index
index_drp # get the index of the above anomalies 

In [ ]:
# above rows are clear anomalies and we can drop them . This is okay since we have more than enough rows to carry out regression

In [ ]:
df3_job.drop(index_drp,axis=0,inplace=True) # drop the rows

In [ ]:
df3_job.reset_index(drop=True,inplace=True)# reset index to keep the shape and indexing 

In [ ]:
df3_job.shape

*** After carrying out the data exploration and creating a data dictionary, below are the columns which has mutiple choice answers in each columns rather than spread out like columns like Q11 or Q13 ***

In [ ]:
cols_mult = ['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q10','Q12_MULTIPLE_CHOICE','Q17','Q18','Q20','Q22','Q23','Q24','Q25','Q26','Q32','Q37','Q39_Part_1','Q39_Part_2','Q40','Q41_Part_1','Q41_Part_2',
'Q43','Q48']

*** lets see how many Nans are there in each of these columns and figure out how to clean them ***

In [ ]:
Ques = []
nan_cols = []
nans = []
# this loop will print number of NaNs in each selected columns 
for col in cols_mult:
    print('#########################################')
    print('Nulls','in',col,'(',df[col].iloc[0],')''=', df3_job[col].isnull().sum())
    if df3_job[col].isnull().sum() >0 :
        nan_cols.append(col)
        nans.append(df3_job[col].isnull().sum())
        Ques.append(df[col].iloc[0])
        
    print(df3_job[col].value_counts(dropna = False))

In [ ]:
# that was pretty big list , lets summarise them by only looking at the columns with NaNs from the earlier subset
pd.DataFrame(list(zip(nan_cols,Ques,nans)),columns= ['Number','Question', 'Number of Nans'])

In [ ]:
df3_job.reset_index(drop=True); # lets reset the index before proceeding

In [ ]:
list(set(cols_mult) - set(nan_cols))

### 'Q6', 'Q2', 'Q3', 'Q1', 'Q4', 'Q7' columns do not have any missing values so I ll leave them alone

### Lets look at the salaries . I see quite a number of outliers . let's investigate 

In [ ]:
df3_job.Q9.quantile(0.01) # looking at the 1 percentile salary 

In [ ]:
df3_job.Q9.quantile(0.995) # looking at the 99.5 percentile salary 

In [ ]:
outlier_index=[]
for i in range(df3_job.shape[0]):
    if ((df3_job.Q9.iloc[i] < df3_job.Q9.quantile(0.01)) | 
        (df3_job.Q9.iloc[i] > df3_job.Q9.quantile(0.995))):
        outlier_index.append(i)
#Let

In [ ]:
len(outlier_index)

In [ ]:
df3_job.shape[0]

In [ ]:
# lets have a brief look at the 
df3_job.iloc[outlier_index,:].Q9.hist(bins =100)
plt.xlabel('Annual Salary')
plt.ylabel('Count')
plt.title('Histogram of outlier salaris')

In [ ]:
# lets look at the sum of NaNs across the rows 
df3_job.iloc[outlier_index,:].isnull().sum(axis=1)

In [ ]:
df3_job.iloc[outlier_index,:].isnull().sum(axis=1).sum()/len(outlier_index)

In [ ]:
df3_job.shape

** There are quite a lot of missing data across the rows for these 122 rows and the salaries are quite extremes which I feel is not correct. This will prevent the model from generalizing. So I decided to drop these rows **

In [ ]:
df3_job.drop(outlier_index,axis=0,inplace=True)

In [ ]:
df3_job.reset_index(inplace=True,drop=True)

### Q5  -  Which best describes your undergraduate major?

In [ ]:
print('Nulls =', df3_job.Q5.isnull().sum())
df3_job.Q5.value_counts(dropna=False)

In [ ]:
df3_job[(df3_job.Q5.isnull())].tail() 

### looking at the data set it is quite evident that we all the NaNs are with people who do not have any formal education past high school . so we can replace Nans with ' No_undergrad'

In [ ]:
df3_job.Q5.fillna(value='No_undergrad',inplace=True)

In [ ]:
df3_job.Q5.isnull().sum()

### Q8  -  How many years of experience do you have in your position?

In [ ]:
df3_job[(df3_job.Q8.isnull())].head(14) 

#### We can safely fill the Q8 column for students with 0-1 years of experience . 

In [ ]:

df3_job.Q8[((df3_job.Q8.isnull()) & (df3_job.Q6 =='Student' ))]= df3_job.Q8[((df3_job.Q8.isnull()) & (df3_job.Q6 =='Student' ))].fillna('0-1')


In [ ]:
df3_job[(df3_job.Q8.isnull())].head(14) 

#### looking at the age , I have chosen experience as below for each of the rows and imputed them 

In [ ]:
values = ['30 +', '10-15','20-25','1-2','1-2']

In [ ]:
df3_job['Q8'].loc[(df3_job.Q8.isnull())] = values # imputing Nans with values

In [ ]:
df3_job[(df3_job.Q8.isnull())]

### Q10  -  Does your current employer incorporate machine learning ?

In [ ]:
df3_job.Q10.value_counts(dropna= False)

### The above NaNs are missed out pretty much in random . lets go deeper by looking at these rows where Q10 is NaN and see if how many other columns are left  unfilled by these people 


In [ ]:
df3_job[df3_job.Q10.isnull()].isnull().sum(axis=1)

In [ ]:
df3_job[df3_job.Q10.isnull()].isnull().sum(axis=1).sum()/118

*** We can clearly see that these 118 Data enthusiasts were not dedicated enough to fill most of the columns. This data do not represent the entire population and we can safely drop them  ***

In [ ]:
df3_job.shape

In [ ]:
df3_job.drop(df3_job[df3_job.Q10.isnull()].index,inplace=True) # dropping the rows
df3_job.shape

In [ ]:
df3_job.reset_index(drop=True,inplace=True) # resetting the index

### Q12  What is the primary tool that you use at work ?

In [ ]:
df3_job.Q12_MULTIPLE_CHOICE.value_counts(dropna=False)

In [ ]:
df3_job[df3_job.Q12_MULTIPLE_CHOICE.isnull()].isnull().sum(axis=1).sum()/639 
# Check across the rows for the average numbers of Nans (across 359 columns) for the NaNs  in Q12 column. 
# We can see that there are high number of Nans across these rows .. 318 in 360! 
 

In [ ]:
# lets look at the data where Q12 is left unfilled 
df3_job[df3_job.Q12_MULTIPLE_CHOICE.isnull()].loc[:,:'Q12_MULTIPLE_CHOICE']

In [ ]:
# I tried encoding the Nans with a separate 'UNKNOWN' category . models were performing much worse . 
# since I had enough data points. I decided to drop these rows
df3_job.drop(df3_job[df3_job.Q12_MULTIPLE_CHOICE.isnull()].index,inplace=True)
df3_job.shape

In [ ]:
df3_job.reset_index(drop=True,inplace=True)

In [ ]:
df3_job.shape

### Q17 	What specific programming language do you use

In [ ]:
df3_job.Q17.value_counts(dropna=False)

## Looking at the data where Q17 is Nan, I could see that most of the Nans has other related columns filled in regarding the programming language of choice etc.

In [ ]:
df3_job[df3_job.Q17.isnull()].Q18.value_counts(dropna=False) # Most of the Nans in Q17 has answered Q18

In [ ]:
# I will fill it with a 'None' category which would mean they did not use any of these languages


In [ ]:
df3_job.Q17.fillna(value='None',inplace=True)
df3_job.reset_index(drop=True,inplace=True)

# Q18 	What programming language would you recommend?

In [ ]:
df3_job.Q18.value_counts(dropna=False)

In [ ]:
# very few Nans in this column and it makes sense to fill the Nans with Mode of the column which is python

In [ ]:
df3_job.Q18.fillna(df3_job.Q18.mode()[0],inplace=True) # using mode as the fillna value

# Q20 	Of the choices that you selected in the previous section ..?


In [ ]:
df3_job.Q20.value_counts(dropna=False)

In [ ]:
# 1882 Nans here . it is makes sense to group these Nans seprately 

In [ ]:
df3_job.Q20.fillna(value='None',inplace=True)

### Q22    --- which specific library was used the most

In [ ]:
df3_job.Q22.value_counts(dropna=False)

In [ ]:
df3_job.Q22.fillna(value='None',inplace=True) # Similar to above

### Q23 Approximately what percent of your time at work do you spend coding ? 

In [ ]:
df3_job.Q23.value_counts(dropna=False)

In [ ]:
df3_job[df3_job.Q23.isnull()].isnull().sum(axis=1)
# most of the Nan rows have sparce entries 

In [ ]:
# dropping these 122 rows 
df3_job.drop(df3_job[df3_job.Q23.isnull()].index,inplace=True)

In [ ]:
df3_job.reset_index(drop=True,inplace=True)

In [ ]:
# lets look at India especially about coders with more than Three hundred thousand USD annual salary


In [ ]:
df3_job[(df3_job['Q9'] > 300000) & (df3_job.Q3 == 'India')].loc[:,'Q9'] # look at these salaries 

In [ ]:
# I see that these annual salaries in USD is not achievable for a business analyst having 2 years experience and aged 
#25, in India . So it is safe to assume that they used their Indian salary instead of US Salary . I am quite certain since I have 
# been working in India . I wll scale it by dividing with 70 ( 1 USD = 70 INR)
val_impute = df3_job[(df3_job['Q9'] > 300000) & (df3_job.Q3 == 'India')].loc[:,'Q9'].values/70
val_impute

In [ ]:
df3_job.loc[df3_job[(df3_job['Q9'] > 300000) & (df3_job.Q3 == 'India')].index,'Q9'] = val_impute # correcting the salary

In [ ]:
# After looking at the data frame, I can see that 95% of these Nans are 20-25 years of age and I expect we can fill the
#Q23 nans with median 
#df3_job.Q23.fillna(value='50% to 74% of my time' ,inplace=True)

#### Working on ordinal columns -- Q23

In [ ]:
df3_job.Q23.value_counts(dropna=False)

In [ ]:
#df3_job_ = df3_job.copy()

In [ ]:
#df3_job = df3_job_.copy()

*** since the above feature is ordinal I would like to keep the order and label encode this column *** 

In [ ]:
mapper = {'50% to 74% of my time':3 ,'25% to 49% of my time' : 2, '1% to 25% of my time' : 1, '100% of my time':5 ,'0% of my time':0,'75% to 99% of my time':4 }

In [ ]:
df3_job.Q23 = df3_job.Q23.map(mapper) # label encoding 

In [ ]:
df3_job.Q23.head()

In [ ]:
df3_job[df3_job.Q23.isnull()].Q3.value_counts() # sanity check 

### Q24  -  Approximately what percent of your time at work or school is spent actively coding?

In [ ]:
df3_job.Q24.value_counts(dropna=False)

In [ ]:
df3_job.Q24.fillna(value='Other',inplace=True) # fill Nans as  'Other' since I could not find any stron reason to do anythin else

### Q25  -  For how many years have you used machine learning methods (at work or in school)?

In [ ]:
df3_job.Q25.value_counts(dropna=False)

In [ ]:
df3_job.Q25.fillna(value='Other',inplace=True)

In [ ]:
df3_job.Q26.value_counts(dropna=False)

In [ ]:
df3_job.shape

In [ ]:
df3_job[df3_job.Q24.isnull()][df3_job.Q25.isnull()][['Q23','Q24','Q25','Q26']].isnull().sum()

In [ ]:

#df3_job= df3_job.drop(df3_job[df3_job.Q25.isnull()].index)

In [ ]:
#df3_job[df3_job.Q24.isnull()][df3_job.Q25.isnull()][['Q23','Q24','Q25','Q26']].fillna('other',inplace=True)

In [ ]:
df3_job.shape

In [ ]:
#df3_job.Q24.fillna(value='1-2 years',inplace=True)

In [ ]:
mapper = {'I have never written code but I want to learn':2,'< 1 year':1,'1-2 years':3,'3-5 years':4,'5-10 years':5,'10-20 years':6,\
         'I have never written code and I do not want to learn':1,'20-30 years':7,'30-40 years':8,'40+ years':9,'Other':0}

In [ ]:
df3_job.Q24.value_counts(dropna=False)

In [ ]:
df3_job.Q24= df3_job.Q24.map(mapper)

In [ ]:
df3_job.Q24.value_counts(dropna=False)

In [ ]:
df3_job.Q24.hist() # good normal distribution

In [ ]:
df3_job.Q25.isnull().sum()

In [ ]:
df3_job.reset_index(drop=True,inplace=True)

In [ ]:
df3_job.Q25.value_counts(dropna=False)

In [ ]:
mapper = {'< 1 year':2,'1-2 years':3,'2-3 years':4,'3-4 years':5,'4-5 years':6,\
         'I have never studied machine learning and I do not plan to':0,'5-10 years':7,'10-15 years':8,'20+ years':9,\
          'I have never studied machine learning but plan to learn in the future':2,'Other':1} 
# used ordinal values in the asending order of experience

In [ ]:
df3_job.Q25= df3_job.Q25.map(mapper) # Label encoding

In [ ]:
df3_job.Q25.isnull().sum()

In [ ]:
df3_job.Q26.value_counts(dropna=False)

In [ ]:
## add more features

In [ ]:
df3_job.Q26.isnull().sum()

In [ ]:
df3_job[df3_job.Q26.isnull()]

In [ ]:
# looking at the data. These 28 people are more likely to be a may be case
df3_job.Q26.fillna(value='Maybe',inplace=True)

In [ ]:
mapper = {'Definitely yes':6,'Probably yes':5,'Maybe':4,'Other':3,'Probably not':2,\
         'Definitely not':1}

In [ ]:
df3_job.Q26= df3_job.Q26.map(mapper)

In [ ]:
df3_job.Q26.isnull().sum()

In [ ]:
df3_job.Q26.astype('int');

In [ ]:
df3_job.Q26= df3_job.Q26.astype('category',);

In [ ]:
df3_job.Q26.dtype

# Feature Selection  and Feature Importance visualisations

#### Why is Feature Selection important ? 

Feature selection is very important in machine learning  because it serves as a fundamental technique to pick the most efficient variables that can be used to predtict the target. Adding to that is the reduction  of the curse of dimensionality or help deal with overfitting. This will eventually help in a lean and efficient model that can be deployed in production efficiently. Further, accurate feature set corresponds to the best bias-variance tradeoff point for the learning algorithm.

#### I will start off with a selected set of features that I had cleaned earlier 

In [ ]:
dataset= df3_job[['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q9','Q10','Q12_MULTIPLE_CHOICE','Q17','Q20','Q22','Q18','Q23','Q24','Q25','Q26']]
dataset.dtypes.value_counts()

In [ ]:
dataset.shape

In [ ]:
dataset.to_csv("dataset.csv") # to be used in Neural network 

In [ ]:
y = dataset.pop('Q9'); # separate the target variable

#### defining a function to find the unbiased fit estimator Adjusted $R^2$

In [ ]:
def adjusted_r2(y_true,y_pred,data):
    r2 = r2_score(y_true,y_pred)
    return(1 - (1-r2)*(data.shape[0]-1)/(data.shape[0]-data.shape[1]-1))

In [ ]:
# change the features to categorical 
dataset.Q23 = dataset.Q23.astype('category')
dataset.Q24 = dataset.Q23.astype('category')
dataset.Q25 = dataset.Q25.astype('category')

*** One hot encoding to ensure that the columns with multiple categories are separated in different columns. I am using pandas get_dummies function to encode the dataframe (will encode only object or categorical type columns). While doing so , to ensure that we get rid of the dummy variable trap, drop_first parameter needs to be set as True.This step will produce k-l features from a column with k categories . ***

In [ ]:
dataset.dtypes.value_counts()

In [ ]:
dataset = pd.get_dummies(dataset,drop_first=True)

In [ ]:
dataset.shape # looks like get_dummies did its work!!

In [ ]:
len(dataset.columns)

In [ ]:
dataset.head() # nice and easy 

*** we have 213 features now . lets see how many is can be removed. I would like to check Lasso,RFE and Random Forest regressor  *** 

In [ ]:
# Train test spliting with test set of 20%
X_train, X_test, y_train , y_test = train_test_split(dataset.values,y, test_size=0.2, random_state=9)

In [ ]:
X_train.shape

In [ ]:
# for Lasso it would help to scale 

In [ ]:
x_scaler = StandardScaler()

In [ ]:
y_scaler = StandardScaler()

In [ ]:
X_train_scaled = x_scaler.fit_transform(X_train)

In [ ]:
X_test_scaled = x_scaler.transform(X_test)

In [ ]:
las = Lasso() # using Lasso regularisation to order the features in the order of importance 

In [ ]:
las.fit(X_train_scaled,y_train)

In [ ]:
pred = las.predict(X_train_scaled)

In [ ]:
adjusted_r2(y_train,pred,X_train) # using adjsted R2 score to get an idea of the fit

In [ ]:
las_importances = las.coef_  # For lasso regression coefficients indicate the importance of the features
las_importances.shape

In [ ]:
indices = np.argsort(abs(las_importances))[::-1] # arg sort will deliver the indices .. absolute value is important 
# as some coeffients may negatively effect the target variable
indices

In [ ]:
feature_lasso =[(dataset.columns[i]) for i in indices ] # get the important features in order

In [ ]:
# Plot the feature importancies of LASSO Regression 
num_to_plot = 15
feature_indices = indices

# Print the feature ranking
print("Feature ranking:")
  
for f in range(num_to_plot):
    print("%d. %s %f " % (f + 1, 
            dataset.columns[indices[f]], 
            las_importances[indices[f]]))
plt.figure(figsize=(20,10))
plt.title(u"Feature Importance")
bars = plt.bar(range(num_to_plot), 
               abs(las_importances[indices[:num_to_plot]]),
       color=([str(i/float(num_to_plot+1)) 
               for i in range(num_to_plot)]),
               align="center")
ticks = plt.xticks(range(num_to_plot), 
                   feature_lasso,rotation=90)
plt.xlim([-1, num_to_plot])
plt.legend(bars, [u''.join(dataset.columns[indices[i]]) 
                  for i in range(num_to_plot)]);
plt.xlabel('Features')
plt.ylabel('Coeffients')

In [ ]:
## explanations

#### using Random forest we can check out feature importance . This  method is based on the Information gain while splitting on important features

In [ ]:
forest = RandomForestRegressor()

In [ ]:
forest.fit(X_train,y_train)

In [ ]:
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1] # sort the features and get index 

In [ ]:
np.argsort(importances)[::-1]

In [ ]:
dataset.columns[indices[0:20]]

In [ ]:
# Plot the feature importancies of the forest
num_to_plot = 15
feature_indices = indices

# Print the feature ranking
print("Feature ranking:")
  
for f in range(num_to_plot):
    print("%d. %s %f " % (f + 1, 
            dataset.columns[indices[f]], 
            importances[indices[f]]))
plt.figure(figsize=(15,5))
plt.title(u"Feature Importance")
bars = plt.bar(range(num_to_plot), 
               importances[indices[:num_to_plot]],
       color=([str(i/float(num_to_plot+1)) 
               for i in range(num_to_plot)]),
               align="center")
ticks = plt.xticks(range(num_to_plot), 
                   dataset.columns[indices[0:num_to_plot]],rotation=90)
plt.xlim([-1, num_to_plot])
plt.legend(bars, [u''.join(dataset.columns[indices[i]]) 
                  for i in range(num_to_plot)]);
plt.xlabel('Features')
plt.ylabel('Importance')

## Discussion on Feature Importance 

** Looking at the feature importances from both Lasso and Random forest regression . we can see that the country where data enthusiast is from, takes up a very high position in the top feature. This correlation is visible from the correlation plot as well. USA dominates both the feature importance results  along with 'We have well established ML methods (i.e., models in production for more than 2 years)' .These makes intuitive sense and we can see that most of the important features are more or less the same ** <br>
Important original attributes are .
1. Country 
2. Age 
3. Use of well established ML models in the company (Q10)
4. Industry of work
5. Use of Visualisations like Matplotlib and Seaborn

# Using  Sequential feature selection from MLXTEND package to sequentially select the features that matters. 
This method is compute expensive <br> reference 
https://pypi.org/project/mlxtend/


In [ ]:
lin_reg = LinearRegression() 
# Sequential feature selection function needs a regressor to check the features sequentially I use the Linear Regressor


In [ ]:
lin_reg.fit(X_train,y_train)

In [ ]:
pred = lin_reg.predict(X_train)

In [ ]:
adjusted_r2(y_train,pred,X_train)

In [ ]:
lin_reg = LinearRegression() 

In [ ]:
#!pip install mlxtend

In [ ]:
# Install mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector

selector = SequentialFeatureSelector(lin_reg, verbose=3, k_features='best', n_jobs=-1,cv=5,scoring = 'r2')

# k_features='best' find the best features from the whole set 

#selector.fit(X_train, y_train)
#Takes almost an hour run time on a google cloud vm with 8 Cores and 30 GB ram !!  So I saved the features as a 
# a data frme and attached with the submission.
# please feel free to uncomment and try it yourself 

In [ ]:
#pd.DataFrame(selector.subsets_).to_csv('SFS_subset.csv')

In [ ]:
#seq_feat_imp = [(dataset.columns[i],i ) for i in selector.k_feature_idx_]


In [ ]:
#pd.DataFrame(seq_feat_imp).to_csv("SFS.csv")

In [ ]:
SFS_selection = pd.read_csv("SFS.csv",header=None,names=['index','SFS_Sel_Features','SFS_Sel_index'])
SFS_selection.drop(index=0,axis=0,inplace=True)

# read the CSV that contains the selected features 

In [ ]:
las.fit(X_train_scaled[:,SFS_selection.SFS_Sel_index],y_train)

In [ ]:
pred = las.predict(X_train_scaled[:,SFS_selection.SFS_Sel_index])

In [ ]:
adjusted_r2(y_train,pred,X_train_scaled)

In [ ]:
X_train_sel = X_train[:,SFS_selection.SFS_Sel_index]
X_test_sel = X_test[:,SFS_selection.SFS_Sel_index]

In [ ]:
X_train_sel.shape 

Sequential feature extractor selected 99 feature and they are not hampering the test score . Moving forward I will use thee selected features for model implementation

### lets check the correlation plot of the selected features . Independence of the features is as important assumtion on a linear model 

In [ ]:
dataset_corr = pd.concat([y,dataset],axis=1) # concatenate the dataset frame with salary so that we can check the correlatiojn 

In [ ]:
# function that will use the data set and thresold to return column names that has correlation more than the threshold
def correlation(dataset,threshold):
    col_corr=set() # set will contains unique values.
    corr_matrix=dataset.corr() #finding the correlation between columns.
    for i in range(len(corr_matrix.columns)): #number of columns
        for j in range(i):
            if abs(corr_matrix.iloc[i,j])>threshold: #checking the correlation between columns.
                colName=corr_matrix.columns[i] #getting the column name
                col_corr.add(colName) #adding the correlated column name heigher than threshold value.
    return col_corr #returning set of column names


In [ ]:
correlated_cols = correlation(dataset_corr,0.7) # call the function and 
correlated_cols

In [ ]:
correlated_cols = set(correlated_cols)

In [ ]:
corr_matrix=dataset_corr.corr()

In [ ]:
corr_matrix['Q9'].abs().sort_values(ascending = False) # checking the columns that are mostly correlated with Salary 

In [ ]:
# Draw the heatmap with the mask and correct aspect ratio
f, ax = plt.subplots(figsize=(20, 15))
sns.heatmap(corr_matrix)
plt.title("Correlation Plot ")

### Lets check if the correlated columns are in the selected list of columns by SFS  and only keep one column from that set 

In [ ]:
len(list(set(SFS_selection.SFS_Sel_Features.values) - correlated_cols))

In [ ]:
len(set(SFS_selection.SFS_Sel_Features)) 

#### Great .. SFS has already done the good job of removing correlated features and keep only one column from the correlated set . We are in good shape to do model implementation

# Model implementation

## 1.  Lasso regression model

In [ ]:
x_scaler = StandardScaler() # Lasso works well with scaled features 

In [ ]:
X_train_sel_scaled = x_scaler.fit_transform(X_train_sel)

In [ ]:
X_test_sel_scaled = x_scaler.transform(X_test_sel)

In [ ]:
lasso_reg = Lasso(random_state=9) # lets try the default lasso parameters we can tune them after getting benchmark

In [ ]:
#Carry out 10 fold cross validation the Ridge regression model
scores_lasso = cross_val_score(n_jobs=-1,estimator=lasso_reg, X=X_train_sel_scaled, y=y_train, cv=10)


In [ ]:
# lets plot the average R2 score and the variance 
sns.lineplot(y=scores_lasso*100,x=range(10))
plt.xlim(0,10)
plt.ylim(0,100)

plt.legend(['Average R2 Score'])
plt.title("Cross validation Score")
plt.xlabel("Folds")
plt.ylabel("R2_score")

In [ ]:
print("Average Accuracy for Gradient Boosting Regressor over 10 folds :",round(scores_lasso.mean() ,2))
print("Varience of the Accuray Gradient Boosting Regressor over 10 folds :",round(scores_lasso.std()*100,2), '%')

In [ ]:
reg_results = {'Lasso': [round(scores_lasso.mean() ,2), round(scores_lasso.std(),2)],\
               'Ridge': [3, 4] ,'GBM': [1, 2], 'KNN_Reg': [3, 4] }

In [ ]:
pd.DataFrame(reg_results, index=['Mean','Std'])

# Discussion on Lasso 

*LASSO (Least Absolute Shrinkage Selector Operator) is a regluarlised linear regression and . It fits a linear regressor which as a penalty term that prevents the Beta coefficients from blowing up . Since Lasso uses L1 distance as the measure of estimator for  Beta values across the features, it does a good job in pruning out irrelevant features. Fast implementation along with the ability to prune out features was the main pro of this algorithim and it was one of the main criterion for choosing this model. On the other hand this model is not much of use in non linear data unless we insert engineered polynomial features.

While going for the cross validation the model accuracy fluctuates across the folds quite a bit. One reason for the variance is due to the data which has very high variability *

# Hyperparameter tuning -Lasso




In [ ]:
%%time

lasso_params = {'alpha':[0.001,0.1,0.5,1,10,20,30,40,50,500 ]}


Lasso_grid = GridSearchCV(lasso_reg, param_grid=lasso_params,cv=10, n_jobs=-1, verbose=1)



In [ ]:
Lasso_grid.fit(X_train_sel_scaled, y_train)# fit the gridsearch instance

In [ ]:
Lasso_grid.best_estimator_

In [ ]:
print("Best R2 score after Grid search " ,round(Lasso_grid.best_score_,2))

In [ ]:
print("mean Variance after Grid search " ,round(Lasso_grid.cv_results_['std_test_score'].mean()*100,2),'%')

In [ ]:
grid_results_l= pd.DataFrame(Lasso_grid.cv_results_)
grid_results_l.index=grid_results_l.params

In [ ]:
grid_results_l.head()

In [ ]:
alphas = lasso_params['alpha']

In [ ]:

sns.lineplot(data = grid_results_l[['mean_test_score','mean_train_score']],x=alphas,y='mean_test_score')
sns.lineplot(data = grid_results_l[['mean_test_score','mean_train_score']],x=alphas,y='mean_train_score')
plt.legend(['Test Accuracy','Train Accuracy'])
plt.title("Bias- Variance Trade off ")
plt.xlabel("Alpha")
plt.ylabel("R2_score")
plt.xlim(0,max(alphas))

## Discussion on Bias variance Trade off

** One of the most important param
Regarding the bias variance trade off for Lasso is aplha - since we have done our due diligence by choosing the best features already there is very little that can be done by Gridsearch . I chose to tune the lasso regularisaton coefficient alpha for bias vaiance analysis . for the best test accuracy , alpha was found to be 10 **

# Model using Ridge Regression

In [2]:
from sklearn.linear_model import Ridge

In [9]:
ridge_reg = Ridge(random_state=9) # use default parameters

In [ ]:
#Carry out 10 fold cross validation the Ridge regression model
scores_ridge = cross_val_score(n_jobs=-1,estimator=ridge_reg, X=X_train_sel_scaled, y=y_train, cv=10)


In [ ]:
sns.lineplot(y=scores_ridge*100,x=range(10))
plt.xlim(0,10)
plt.ylim(0,100)

plt.legend(['Average R2 Score'])
plt.title("Cross validation Score")
plt.xlabel("Folds")
plt.ylabel("R2_score")

In [ ]:
print("Average Accuracy for Gradient Boosting Regressor over 10 folds :",round(scores_ridge.mean() ,2))
print("Varience of the Accuray Gradient Boosting Regressor over 10 folds :",round(scores_ridge.std()*100,2), '%')

# Discussion on Ridge Regression

*Similar to Lasso  regluarlised linear regression and . It fits a linear regressor which as a penalty term that prevents the Beta coefficients from blowing up . Here the algorithm uses  L2 distance as the measure of estimator for  Beta values across the features. Howver, it will not completely eliminate the feature like Lasso since the distance measure boundary is circular in nature .  

Fast implementation along with the ability to keep beta values in check features was the main pro of this algorithim and it was one of the main criterion for choosing this model. simlar to Lasso this model is not much of use in non linear data unless we insert engineered polynomial features.

While going for the cross validation the model accuracy fluctuates across the folds quite a bit. One reason for the variance is due to the data which has very high variability.
*

# hyper parameter tuning Ridge 

In [ ]:
%%time

ridge_params = {'alpha':[0.1,0.5,0.8,.9,0.99,2,3,4,5,6,20,30,40,50,500]}


ridge_grid = GridSearchCV(ridge_reg, param_grid=ridge_params,cv=10, n_jobs=-1, verbose=1)



In [ ]:
ridge_grid.fit(X_train_sel_scaled, y_train)# fit the gridsearch instance

In [ ]:
ridge_grid.best_estimator_

In [ ]:
print("Best R2 score after Grid search " ,round(ridge_grid.best_score_,2))

In [ ]:
print("mean Variance after Grid search " ,round(ridge_grid.cv_results_['std_test_score'].mean()*100,2),'%')

In [ ]:
grid_results= pd.DataFrame(ridge_grid.cv_results_)
grid_results.index=grid_results.param_alpha

In [ ]:
# plotting Bias Variance Trade off by looking at the ERRORS 
(1-grid_results[['mean_test_score','mean_train_score']]).plot()
plt.legend(['Test Error','Train Error'])
plt.title("Bias- Variance Trade off ")
plt.xlabel("Alpha")
plt.ylabel("R2_score")
#plt.ylim(0,1.0)



## Discussion on Bias Variance Tradeoff

** One of the most important param Regarding the bias variance trade off for Ridge regression is still alpha .howver the algo uses L2 norm - since we have done our due diligence by choosing the best features already there is very little that can be done by Gridsearch . I chose to tune the Ridge regularisaton coefficient alpha for bias vaiance analysis . for the best test accuracy , alpha was found to be 50 **

## Gradient Boosting Model

In [5]:
gbm_reg = GradientBoostingRegressor(random_state=9) # default gradent boosting parameters

NameError: name 'GradientBoostingRegressor' is not defined

In [ ]:
gbm_reg.fit(X_train_sel, y_train)

In [ ]:

scores_gbm = cross_val_score(n_jobs=-1,estimator=gbm_reg, X=X_train_sel, y=y_train, cv=10) # cross validation


In [ ]:
sns.lineplot(y=scores_gbm*100,x=range(10),estimator='mean')
plt.xlim(0,10)
plt.ylim(0,100)

plt.legend(['Average R2 Score'])
plt.title("Cross validation")
plt.xlabel("Folds")
plt.ylabel("R2_score")

In [ ]:
print("Average Accuracy for Gradient Boosting Regressor over 10 folds :",round(scores_gbm.mean(),2),)
print("Varience of the Accuray Gradient Boosting Regressor over 10 folds :",round(scores_gbm.std()*100,2), '%')

# Discussion on GBM 

*Gradient boosting algorithm is an ensemble method which uses a number of boosted trees and aggregate them to come to a prediction. Decision trees formed one at a time, where each new tree helps to correct errors made by previously trained tree. Since this algo is based on decision trees, they are inherently better at handling missing values and do not need any scaling. 

GBDT training generally takes longer because of the fact that trees are built sequentially and chances of over fitting is very high if not kept a check . Boosting and ensemble methods were best performers in Kaggle competitions and they help up erk out maxmimum information from the given data set. This method also performs better even if the data is quite non linear

Here we can see that the ensemble method has much less variance compared to previous methods.* 


# Hyper parameter tuning -GBM 

In [ ]:

gbm_params = {'n_estimators':range(100,350,20)}  # parameter grid

#scoring = {'ERROR': make_scorer(mean_squared_error), 'Accuracy': make_scorer(r2_score)}

gbm_grid = GridSearchCV(gbm_reg, param_grid=gbm_params,cv=10, n_jobs=-1, verbose=1)



In [ ]:
%%time

gbm_grid.fit(X_train_sel, y_train)# fit the gridsearch instance

In [ ]:
gbm_grid.best_estimator_

In [ ]:
print("Best R2 score after Grid search " ,round(gbm_grid.best_score_,2))

In [ ]:
print("mean Variance after Grid search " ,round(gbm_grid.cv_results_['std_test_score'].mean()*100,2),'%')

In [ ]:
grid_results

In [ ]:
grid_results= pd.DataFrame(gbm_grid.cv_results_) # convert to DF
grid_results.index=grid_results.param_n_estimators # get the index 

In [ ]:
# plotting Bias Variance Trade off
(1-grid_results[['mean_test_score','mean_train_score']]).plot()
plt.legend(['Test Error','Train Error'])
plt.title("Bias- Variance Trade off ")
plt.xlabel("N_Estimators")
plt.ylabel("R2_score")


## Discussion on Bias Variance Tradeoff

** In GBM I chose to tune the number of estimators (number of trees) that are formed . After exploring the paramter grid, it was found that 300 is the number that optimizes test error as well as the training error. ** 


# KNNregressoor

In [ ]:
x_scaler = StandardScaler() # create standard scaler object for knn since it is distance based. 

In [ ]:
X_train_sel_scaled = x_scaler.fit_transform(X_train_sel)  # fit the scaler and transform

In [ ]:
X_test_sel_scaled = x_scaler.transform(X_test_sel) # tranform using the fitted scaler

In [ ]:
knn_reg = KNeighborsRegressor()

In [ ]:
knn_reg

In [ ]:

scores_knn = cross_val_score(n_jobs=-1,estimator=knn_reg, X=X_train_sel_scaled, y=y_train, cv=10)


In [ ]:
sns.lineplot(y=scores_knn*100,x=range(10),estimator='mean')
plt.xlim(0,10)
plt.ylim(0,100)

plt.legend(['Average R2 Score'])
plt.title("Cross validation")
plt.xlabel("Folds")
plt.ylabel("R2_score")

In [ ]:
print("Average Accuracy for Gradient Boosting Regressor over 10 folds :",round(scores_knn.mean(),2), )
print("Varience of the Accuray Gradient Boosting Regressor over 10 folds :",round(scores_knn.std()*100,2), '%')

# Discussion on KNN 

KNNRegressor is based on the K nearest neighbour princile.It is a non parametric approach.  The target is predicted by local interpolation of the targets associated of the nearest neighbors in the training set.

Advantages are easy to code and understand, and scales up to big data.Disadvantages are sensitive to data and tuning.I wanted to try out a different model to our dataset to see if knn based model will do any good . 

In our case I feel that knn is not a great choice as we can clearly see that the r2 score is way lower that others

# hyper parameter tuning KNN 

In [ ]:
%%time

knn_params = {'n_neighbors':range(5,50,10)}

knn_grid = GridSearchCV(knn_reg, param_grid=knn_params,cv=10, n_jobs=-1, verbose=1)



In [ ]:
knn_grid.fit(X_train_sel_scaled, y_train)# fit the gridsearch instance

In [ ]:
knn_grid.best_estimator_

In [ ]:
knn_grid.best_score_

In [ ]:
grid_results= pd.DataFrame(knn_grid.cv_results_)

In [ ]:
grid_results.index=grid_results.param_n_neighbors

In [ ]:
# plotting Bias Variance Trade off
(1-grid_results[['mean_test_score','mean_train_score']]).plot()
plt.legend(['Test Error','Train Error'])
plt.title("Bias- Variance Trade off ")
plt.xlabel("N_Estimators")
plt.ylabel("R2_score")



## Discussion on Bias Variance Tradeoff

** In Kneighboursregressor I chose to tune the number of nearest neighbors . After exploring the paramter grid, it was found that 30 is the number that optimizes test error as well as the training error . 


# comparision between models

In [ ]:
reg_results = {'Lasso': [round(scores_lasso.mean() ,2), round(scores_lasso.std(),2)],\
               'Ridge': [round(scores_ridge.mean() ,2), round(scores_ridge.std() ,2)] ,
               
               'GBM': [round(scores_gbm.mean(), 2),round(scores_gbm.std(), 2)], 'KNN_Reg': [round(scores_knn.mean(),2), round(scores_knn.std(),2)] }

In [ ]:
comp = pd.DataFrame(reg_results, index=['Mean','Std'])

In [ ]:
#comp.loc['Variance',:]  = comp.loc['Variance',:].apply(np.square). 
#Did not convert STD to Variance since the data pointes were very small to visulize side by side with accuracy 

In [ ]:

comp.plot(kind='bar')
plt.xlabel("Metrics")
plt.title("Comparision between different models - Mean accuracy and Variance")
plt.ylim(0,1)

# Choosing optimal Model

### I chose the gradient boosting model as it had the best r2_score and less variance as per the graph shown above

In [ ]:
gbm_reg = GradientBoostingRegressor(n_estimators=300,random_state=9) # using the best model parameters

In [ ]:
gbm_reg.fit(X_train_sel, y_train)

In [ ]:
pred_tr = gbm_reg.predict(X_train_sel)

In [ ]:
pred_ts = gbm_reg.predict(X_test_sel)

In [ ]:
print ('Mean Squred error from Train set', 928910250.089)

In [ ]:
round(mean_squared_error(y_train,pred_tr),3)

In [ ]:
print("Final Mean Squred error =",round(mean_squared_error(y_test,pred_ts),3))

In [ ]:
print('Difference in train and test mean squared error = ' ,1264952288.319 - 917209405.257)

In [ ]:
print("Final Model Coefficient of determination Train Set =",round(adjusted_r2(y_train,pred_tr,X_train_sel),3))

In [ ]:
print("Final Model Coefficient of determination Test Set =",round(adjusted_r2(y_test,pred_ts,X_test_sel),3))

# Discussion on the selected model


Given the inherent issues with the data set and the subset of the problem I was looking at, I feel that explaining more than 60% of the variance is an indication of a good model. Gradient boosting model showed slightly higher R2 score for the test set I feel that since our data has quite a lot outliers , MSE gets biased with these values arne swing to  extremas.So in this assignment I primarily made use of R2 Score . Even R2Score is biased and I had defined a function to compute Adjusted R2 score. Howver, Adjsted R2 score will differ from R2 score based on the number of features and the sample size.Adjusted R2 Score is  a quite useful metric while selecting features.  R2 Score is a measure of the explained variance.In other words, how much variance of the data set is explaned by the model. It takes values from 0 to 1 with 0 being the worst. R2 score will not decrease with the number of predictors added. Adjusted R2 Score fixes this issue by penalizing the addition of a new feature. 

In our case the slightly higher Test R2 score can be attributed to the difference in distribution of the train and test set and the sparcity and randomness in the the dataset. The model does not look to be overfitting since the test set performed well with model created. I feel that it is slightly underfit or the inherent Bayes error (irremovable error) in the dataset is quite high. The suspicion on the Bayes error is quite valid since almost 6 regressor models and Neural Network model gave similary metric of model fit 


However, I feel that the model could be improved much more  using below mentiond methods 

1. Spend more time on adding more relevant features and use feature selection 
2. Better feature engineering - polynomial features to be introduced 
3. Better utilisation of other text fields that was dropped to make data set simpler
4. Conduct Gridsearch on more parameters for the same regressor

# Thank you very much for the support during the course and relevant assignments. Happy holidays !!